<a href="https://colab.research.google.com/github/byui-cse/cse480-notebooks/blob/master/08_1_Designing_Turing_Machines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Designing Turing Machines

1. Look at Turing Machines historically, abstractly and concretely.
2. Look at a couple alternatives to Jove.

## TODO Explore

Resources you might find helpful:

[This Turing Machine Simulator](http://morphett.info/turing/turing.html)

The Java-based program called [JFLAP](http://www.jflap.org)

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import sys

# -- Detect if in Own Install or in Colab
try:
    import google.colab
    OWN_INSTALL = False
except:
    OWN_INSTALL = True
    
if OWN_INSTALL:
  sys.path[0:0] = ['../../../../..',  '../../../../../3rdparty',  
                   '../../../..',  '../../../../3rdparty',  
                   '../../..',     '../../../3rdparty', 
                   '../..',        '../../3rdparty',
                   '..',           '../3rdparty',
                   '../Jove',      '../Jove/jove']

else: # In colab
  ! if [ ! -d Jove ]; then git clone https://github.com/ganeshutah/Jove Jove; fi
  sys.path.append('./Jove')
  sys.path.append('./Jove/jove')

# -- common imports --
from jove.Def_TM     import *
from jove.Def_md2mc  import *
from jove.DotBashers import *
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Before looking at a couple of simple DTM designs, let's abbreviate:

* cs = Current State
* sr = Symbol Read
* sw = Symbol Written
* md = Move Direction
* ns = Next State

(What is the language of this TM?)

In [ ]:
dotObj_tm(md2mc('''
  TM
!!cs  sr  sw   md   ns
  I0 : x ; x , R -> S1
  S1 : 0 ; 0 , R -> S1
  S1 : 1 ; 1 , R -> S1
  S1 : x ; x , L -> S2
  S2 : 1 ; 0 , L -> S2
  S2 : x ; 1 , S -> Fhalt
  S2 : 0 ; 1 , L -> S3
  S3 : x ; x , S -> Fhalt
  S3 : 0 ; 0 , L -> S3
  S3 : 1 ; 1 , L -> S3
'''), FuseEdges=True)

This TM recognizes the language of all strings over $\Sigma = \{0, 1\}$ that are "marked" repeats:

$L_{w\#w} = \{w\#w\ \vert\ w \in \Sigma^* \land \# \in \Gamma\}$

In [ ]:
dotObj_tm(md2mc('''
  TM
!!cs  sr  sw   md   ns
  I0 : 0 ; X , R -> S1
  I0 : # ; # , R -> S5
  I0 : 1 ; Y , R -> S7
  S1 : # ; # , R -> S2
  S1 : 0 ; 0 , R -> S1
  S1 : 1 ; 1 , R -> S1
  S2 : 0 ; X , L -> S3
  S2 : X ; X , R -> S2
  S2 : Y ; Y , R -> S2
  S3 : # ; # , L -> S4
  S3 : Y ; Y , L -> S3
  S3 : X ; X , L -> S3
  S4 : Y ; Y , R -> S0
  S4 : X ; X , R -> S0
  S4 : 1 ; 1 , L -> S4
  S4 : 0 ; 0 , L -> S4
  S5 : B ; B , R -> F6
  S5 : Y ; Y , R -> S5
  S5 : X ; X , R -> S5
  S7 : # ; # , R -> S8
  S7 : 1 ; 1 , R -> S7
  S7 : 0 ; 0 , R -> S7
  S8 : 1 ; Y , L -> S9
  S8 : Y ; Y , R -> S8
  S8 : X ; X , R -> S8
  S9 : # ; # , L -> SA
  S9 : X ; X , L -> S9
  S9 : Y ; Y , L -> S9
  SA : X ; X , R -> S0
  SA : Y ; Y , R -> S0
  SA : 1 ; 1 , L -> SA
  SA : 0 ; 0 , L -> SA
'''), FuseEdges=True)

In [ ]:
# Write this string to a file named wpoundw.jff and read it into JFLAP
# for comparison:
'''
<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!--Created with JFLAP 4.0b14.-->
<structure>&#13;
	<type>turing</type>&#13;
	<!--The list of states.-->&#13;
	<state id="0">&#13;
		<x>141.0</x>&#13;
		<y>278.0</y>&#13;
		<initial/>&#13;
	</state>&#13;
	<state id="1">&#13;
		<x>289.0</x>&#13;
		<y>130.0</y>&#13;
	</state>&#13;
	<state id="2">&#13;
		<x>469.0</x>&#13;
		<y>146.0</y>&#13;
	</state>&#13;
	<state id="3">&#13;
		<x>630.0</x>&#13;
		<y>223.0</y>&#13;
	</state>&#13;
	<state id="4">&#13;
		<x>787.0</x>&#13;
		<y>258.0</y>&#13;
	</state>&#13;
	<state id="5">&#13;
		<x>292.0</x>&#13;
		<y>236.0</y>&#13;
	</state>&#13;
	<state id="6">&#13;
		<x>472.0</x>&#13;
		<y>222.0</y>&#13;
		<final/>&#13;
	</state>&#13;
	<state id="7">&#13;
		<x>297.0</x>&#13;
		<y>407.0</y>&#13;
	</state>&#13;
	<state id="8">&#13;
		<x>463.0</x>&#13;
		<y>443.0</y>&#13;
	</state>&#13;
	<state id="9">&#13;
		<x>630.0</x>&#13;
		<y>443.0</y>&#13;
	</state>&#13;
	<state id="10">&#13;
		<x>790.0</x>&#13;
		<y>367.0</y>&#13;
	</state>&#13;
	<!--The list of transitions.-->&#13;
	<transition>&#13;
		<from>2</from>&#13;
		<to>3</to>&#13;
		<read>0</read>&#13;
		<write>X</write>&#13;
		<move>L</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>8</from>&#13;
		<to>9</to>&#13;
		<read>1</read>&#13;
		<write>Y</write>&#13;
		<move>L</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>7</from>&#13;
		<to>8</to>&#13;
		<read>#</read>&#13;
		<write>#</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>1</from>&#13;
		<to>2</to>&#13;
		<read>#</read>&#13;
		<write>#</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>0</from>&#13;
		<to>1</to>&#13;
		<read>0</read>&#13;
		<write>X</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>3</from>&#13;
		<to>4</to>&#13;
		<read>#</read>&#13;
		<write>#</write>&#13;
		<move>L</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>9</from>&#13;
		<to>10</to>&#13;
		<read>#</read>&#13;
		<write>#</write>&#13;
		<move>L</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>10</from>&#13;
		<to>0</to>&#13;
		<read>X</read>&#13;
		<write>X</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>10</from>&#13;
		<to>0</to>&#13;
		<read>Y</read>&#13;
		<write>Y</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>4</from>&#13;
		<to>0</to>&#13;
		<read>Y</read>&#13;
		<write>Y</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>4</from>&#13;
		<to>0</to>&#13;
		<read>X</read>&#13;
		<write>X</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>5</from>&#13;
		<to>6</to>&#13;
		<read>B</read>&#13;
		<write>B</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>10</from>&#13;
		<to>10</to>&#13;
		<read>1</read>&#13;
		<write>1</write>&#13;
		<move>L</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>10</from>&#13;
		<to>10</to>&#13;
		<read>0</read>&#13;
		<write>0</write>&#13;
		<move>L</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>9</from>&#13;
		<to>9</to>&#13;
		<read>X</read>&#13;
		<write>X</write>&#13;
		<move>L</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>9</from>&#13;
		<to>9</to>&#13;
		<read>Y</read>&#13;
		<write>Y</write>&#13;
		<move>L</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>3</from>&#13;
		<to>3</to>&#13;
		<read>Y</read>&#13;
		<write>Y</write>&#13;
		<move>L</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>3</from>&#13;
		<to>3</to>&#13;
		<read>X</read>&#13;
		<write>X</write>&#13;
		<move>L</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>4</from>&#13;
		<to>4</to>&#13;
		<read>1</read>&#13;
		<write>1</write>&#13;
		<move>L</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>4</from>&#13;
		<to>4</to>&#13;
		<read>0</read>&#13;
		<write>0</write>&#13;
		<move>L</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>8</from>&#13;
		<to>8</to>&#13;
		<read>Y</read>&#13;
		<write>Y</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>8</from>&#13;
		<to>8</to>&#13;
		<read>X</read>&#13;
		<write>X</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>7</from>&#13;
		<to>7</to>&#13;
		<read>1</read>&#13;
		<write>1</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>2</from>&#13;
		<to>2</to>&#13;
		<read>X</read>&#13;
		<write>X</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>0</from>&#13;
		<to>5</to>&#13;
		<read>#</read>&#13;
		<write>#</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>2</from>&#13;
		<to>2</to>&#13;
		<read>Y</read>&#13;
		<write>Y</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>1</from>&#13;
		<to>1</to>&#13;
		<read>0</read>&#13;
		<write>0</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>1</from>&#13;
		<to>1</to>&#13;
		<read>1</read>&#13;
		<write>1</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>5</from>&#13;
		<to>5</to>&#13;
		<read>Y</read>&#13;
		<write>Y</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>5</from>&#13;
		<to>5</to>&#13;
		<read>X</read>&#13;
		<write>X</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>7</from>&#13;
		<to>7</to>&#13;
		<read>0</read>&#13;
		<write>0</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
	<transition>&#13;
		<from>0</from>&#13;
		<to>7</to>&#13;
		<read>1</read>&#13;
		<write>Y</write>&#13;
		<move>R</move>&#13;
	</transition>&#13;
</structure>
'''